In [ ]:
# read shp files
# rename columns 
# merge into one 
# merge into the datafile



In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [ ]:
# clean up: 

def dropstuff(df):
    df = df.drop(['DN', 'glaceir', 'layer', 'path', 'Name', 'layer_2', 'path_2', 'area_m', 
             'perimeter', 'glaciersum', 'geometry'], axis = 1)
    return df

In [ ]:
# read all .shp files

folder = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/shapes/permafrost/'

permafrost4 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost4.shp')).rename(columns={"permafrost": "permafrost4", "permafro_1": "permafrost4_frac"})
permafrost3 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost3.shp')).rename(columns={"permafrost": "permafrost3", "permafro_1": "permafrost3_frac"})
permafrost2 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost2.shp')).rename(columns={"permafrost": "permafrost2", "permafro_1": "permafrost2_frac"})
permafrost1 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost1.shp')).rename(columns={"permafrost": "permafrost1", "permafro_1": "permafrost1_frac"})

In [ ]:
# clean up 
permafrost4 = dropstuff(permafrost4)
permafrost3 = dropstuff(permafrost3)
permafrost2 = dropstuff(permafrost2)
permafrost1 = dropstuff(permafrost1)

In [ ]:
# merge into the dfclim data 

folder = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/ml_model/data_current/'

dfclim = pd.read_csv(folder + 'dfclim_3aug_rainydays.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [ ]:
dfclim1 = dfclim.merge(permafrost1, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost2, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost3, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost4, on =['x_centroid', 'y_centroid'], how = 'left')

In [ ]:
dfclim1 = dfclim1.drop_duplicates()

# dfclim1.to_csv('out/dfclim_permafrost9aug.csv')

### continious permafrost (type4) yes/no 